### Substitution cipher

In [1]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'
key =      'jsuyfhkpicomxrqatlbvznewgd'


def substitute(text, substitute_what, substitute_by):
    result = ''
    for symbol in text.lower():
        if symbol in substitute_what:
            result += substitute_by[substitute_what.index(symbol)]
        else:
            result += symbol

    return result


def encode(plaintext):
    return substitute(plaintext, alphabet, key)


def decode(ciphertext):
    return substitute(ciphertext, key, alphabet)


message = 'the quick brown fox jumps over the lazy dog'
code = encode(message)
print(code)
print(decode(code))

vpf tziuo slqer hqw czxab qnfl vpf mjdg yqk
the quick brown fox jumps over the lazy dog


### One-time Pad

In [2]:
from itertools import product


def encode(plain_text, private_key):
    assert plain_text in {0, 1} and private_key in {0, 1}
    return plain_text ^ private_key


def decode(cypher_text, private_key):
    assert cypher_text in {0, 1} and private_key in {0, 1}
    return cypher_text ^ private_key


for plaintext, private_key in product({0, 1}, repeat=2):
    ciphertext = encode(plaintext, private_key)
    print(f'key: {private_key}, '
          f'plaintext: {plaintext}, '
          f'ciphertext: {ciphertext}, '
          f'decoded: {decode(ciphertext, private_key)}')

key: 0, plaintext: 0, ciphertext: 0, decoded: 0
key: 1, plaintext: 0, ciphertext: 1, decoded: 0
key: 0, plaintext: 1, ciphertext: 1, decoded: 1
key: 1, plaintext: 1, ciphertext: 0, decoded: 1


In [3]:
def to_hex(plain_text):
    hex_codes = []
    for symbol in plain_text:
        hex_code = hex(ord(symbol)).replace('0x', '')
        if len(hex_code) == 1:
            hex_code = '0' + hex_code
        hex_codes.append(hex_code)
    return ''.join(hex_codes)


def to_str(hex_code):
    if hex_code:
        return chr(int(hex_code[:2], base=16)) + to_str(hex_code[2:])
    return ''


message = 'Hello World'
print(f'hex of {message} is: {to_hex(message)}')

code = '736f6d65206d657373616765'
print(f'str of {code} is: {to_str(code)}')

hex of Hello World is: 48656c6c6f20576f726c64
str of 736f6d65206d657373616765 is: some message


In [4]:
def bitwise_xor(first_text, second_text):
    assert len(first_text) == len(second_text)
    return ''.join(format(int(s1, 16) ^ int(s2, 16), '01x')
                   for s1, s2 in zip(first_text, second_text))


message = 'secret message'
key =     'my secret keys'
print(f'hex of {message} is: {to_hex(message)}')
print(to_hex(key))

ciphertext = bitwise_xor(to_hex(message), to_hex(key))
print('ciphertext:', ciphertext)

recovered_message = to_str(bitwise_xor(ciphertext, to_hex(key)))
print('recovered message:', recovered_message)

hex of secret message is: 736563726574206d657373616765
6d7920736563726574206b657973
ciphertext: 1e1c430100175208115318041e16
recovered message: secret message


In [7]:
def try_guessing_substring(substring, message_length, xor_messages):
    good_guesses = []
    for pos in range(message_length - len(substring) + 1):
        guess = to_hex(chr(0) * pos + substring +
                       chr(0) * (message_length - len(substring) - pos))
        other_message_part = to_str(
            bitwise_xor(guess, xor_messages)
        )[pos:pos + len(substring)]
        good_guess = True
        for i in range(len(other_message_part)):
            if not other_message_part[i].isalpha() and \
                    not other_message_part[i].isspace():
                good_guess = False
                break
        if good_guess:
            good_guesses.append((guess, pos, other_message_part))

    print('Good guesses:')
    for guess in good_guesses:
        print(f'position: {guess[1]}, '
              f'one message part: \"{substring}\", '
              f'another message part: \"{guess[2]}\"')

message1 = 'steal the secret'
message2 = 'the boy the girl'
xor_messages = bitwise_xor(to_hex(message1), to_hex(message2))
try_guessing_substring(' the ', len(message1), xor_messages)

Good guesses:
position: 5, one message part: " the ", another message part: "oy th"
position: 7, one message part: " the ", another message part: "he se"


### RSA

In [13]:
# Alice -> Bob, Bob generates a public key and a private key

# Bob generates 2 big prime numbers p and q and multiplies them to get n
from math import gcd
p, q = 5915587277, 2860486313

n = p * q
print(f'n={n}')

# Bob computes φ(n) = (p - 1) * (q - 1)
phi = (p - 1) * (q - 1)
print(f'φ(n)={phi}')

# Bob chooses an integer e such that 1 < e < φ(n) and gcd(e, φ(n)) = 1; that is, e and φ(n) are coprime
e = 3
print(f'e={e}')

# Public key: (n, e)
print(f'Public key: (n={n}, e={e})')

# Private key: (n, d)
d = pow(e, -1, phi)
print(f'd={d}')
assert d * e % phi == 1

# Encode msg
def encode(m):
    assert 0 <= m < n
    return pow(m, e, n)


message = 92616855427
print(f'message: {message}')
cipher_text = encode(message)

# Decode msg
def decode(c):
    return pow(c, d, n)


print(f'decoded: {decode(cipher_text)}')

n=16921456439215439701
φ(n)=16921456430439366112
e=3
Public key: (n=16921456439215439701, e=3)
d=11280970953626244075
message: 92616855427
decoded: 92616855427
